In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import os
from freezegun import freeze_time
from enum import Enum

from src.supabase_client import get_user
from src.email_manager import mock_send_email
from src.lambda_function import run_weekly_update_process, run_mid_week_update_process
from src.supabase_client import mock_upsert_training_week_update, mock_upsert_training_week_with_coaching
from src.supabase_client import list_users
from src.types.user_row import UserRow
from src.auth_manager import get_strava_client

In [30]:
class UserId(str, Enum):
    JAMIE = os.environ["JAMIES_ATHLETE_ID"]
    JARED = os.environ["JAREDS_ATHLETE_ID"]

In [31]:
def run_weekly_update_process_wrapped(
    mock_user: UserRow,
    date_str: str = "2024-08-25"
):
    @freeze_time(f"{date_str} 20:00:00")
    def run_weekly_update_process_freezed(mock_user: UserRow):
        run_weekly_update_process(
            user=mock_user,
            upsert_fn=mock_upsert_training_week_with_coaching,
            email_fn=mock_send_email,
        )
    get_strava_client(mock_user.athlete_id)
    run_weekly_update_process_freezed(mock_user)


def run_mid_week_update_process_wrapped(
    mock_user: UserRow, 
    date_str: str = "2024-08-27"
):

    @freeze_time(f"{date_str} 20:00:00")
    def run_mid_week_update_process_freezed(mock_user: UserRow):
        run_mid_week_update_process(
            user=mock_user,
            upsert_fn=mock_upsert_training_week_update,
            email_fn=mock_send_email,
        )

    get_strava_client(mock_user.athlete_id)
    run_mid_week_update_process_freezed(mock_user)

# Test with Jamie

In [32]:
mock_user = get_user(UserId.JAMIE.value)
run_weekly_update_process_wrapped(mock_user)

athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
{
    "athlete_id": 98390356,
    "planning": "Starting with the long run, we'll schedule it on Saturday with a distance of 16 miles. This leaves us with 29 to 31 miles to cover during the rest of the week. \n\nFor continuous mid-week workouts, we'll plan a speed workout on Wednesday for a total of 7 miles including warm-up, cool-down, and intervals at 10k pace such as 4x1 mile repeats. \n\nThis leaves us with 22 to 24 miles to distribute across the remaining days. \n\nSunday is typically a rest day or very light running day, so a rest day could be beneficial here. We'll aim for easy runs of 5 miles on Monday and Tuesday, a moderate run of 6 miles on Thursday, a light 3-mile easy run on Friday, and the long run on Saturday. Revisiting the numbers:\n- Monday: 5 miles (easy) \n- Tuesday: 5 miles (easy) \n- Wednesday: Speed workout including 4x1 mile 

In [36]:
mock_user = get_user(UserId.JAMIE.value)
run_mid_week_update_process_wrapped(mock_user)

athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
{
    "athlete_id": 98390356,
    "activities": [
        {
            "date": "Monday, August 26, 2024",
            "distance_in_miles": 0.0,
            "elevation_gain_in_feet": 0.0,
            "pace_minutes_per_mile": NaN
        },
        {
            "date": "Tuesday, August 27, 2024",
            "distance_in_miles": 10.04,
            "elevation_gain_in_feet": 213.25,
            "pace_minutes_per_mile": 9.56
        }
    ],
    "training_week": [
        {
            "day": "mon",
            "session_type": "easy run",
            "distance": 5.0,
            "notes": "Easy pace to allow muscle recovery."
        },
        {
            "day": "tues",
            "session_type": "moderate run",
            "distance": 7.0,
            "notes": "Run at a steady and comfortable pace, slight push on effort."
        },
        {
    